In [15]:
%load_ext autoreload

In [20]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    b_w = [(0,6), (6,12)]
    a_w = [(0,4), (4,8), (8,12)]
    before_windows = b_w
    after_windows = a_w
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [12]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=True, lab_parts=(0,15))

Loading med data...
Generate med data....


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:51: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ROW_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")


KeyboardInterrupt: 

In [ ]:
m_labs = m_labs.reset_index().drop(columns=["index"])
m_labs

,HADM_ID,ADMITTIME,CHARTTIME,VALUENUM,ITEMID,ADMISSION_LOCATION,ADMISSION_TYPE,AGE,DEATHTIME,DIAGNOSIS,...,LANGUAGE,MARITAL_STATUS,RELIGION,SUBJECT_ID,VALUE,VALUEUOM,ROW_ID_x,TABLE,LabTimeFromAdmit,hours_in
0,100003.0,2150-04-17 15:34:00,2150-04-17 18:00:00,68.0,Glucose,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,68.0,NaN,12554668,CHARTEVENTS,0 days 02:26:00,2.433333
1,100003.0,2150-04-17 15:34:00,2150-04-17 18:41:00,1.2,Creatinine,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,1.2,mg/dL,12554679,CHARTEVENTS,0 days 03:07:00,3.116667
2,100003.0,2150-04-17 15:34:00,2150-04-17 18:41:00,1.9,Magnesium,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,1.9,mg/dL,12554681,CHARTEVENTS,0 days 03:07:00,3.116667
3,100003.0,2150-04-17 15:34:00,2150-04-17 18:41:00,7.1,Hemoglobin,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,7.1,g/dl,12554675,CHARTEVENTS,0 days 03:07:00,3.116667
4,100003.0,2150-04-17 15:34:00,2150-04-17 18:41:00,7.5,Calcium,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,7.5,mg/dL,12554684,CHARTEVENTS,0 days 03:07:00,3.116667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006503,199948.0,2102-02-22 13:28:00,2102-02-27 05:25:00,11.6,Hemoglobin,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,11.6,g/dL,21124826,LABEVENTS,4 days 15:57:00,111.950000
2006504,199948.0,2102-02-22 13:28:00,2102-02-27 05:25:00,34.2,Hematocrit,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,34.2,%,21124825,LABEVENTS,4 days 15:57:00,111.950000
2006505,199948.0,2102-02-22 13:28:00,2102-02-27 05:25:00,78.0,Glucose,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,78,mg/dL,21124819,LABEVENTS,4 days 15:57:00,111.950000
2006506,199948.0,2102-02-22 13:28:00,2102-02-27 05:25:00,135.0,Sodium,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,135,mEq/L,21124823,LABEVENTS,4 days 15:57:00,111.950000


In [ ]:
m_labs.ITEMID.value_counts()

Glucose                      297998
Sodium                       172844
Hematocrit                   172798
Hemoglobin                   148820
Creatinine                   147765
Potassium                    147022
Platelets                    138446
Magnesium                    136690
Calcium                      122289
PTT                           93917
Prothrombin time INR          88597
Red blood cell                88435
Calcium ionized               63639
Bilirubin                     45566
Alanine aminotransferase      39515
Asparate aminotransferase     39489
Lactic acid                   24051
Albumin                       23741
Lipase                         6468
Amylase                        4679
Uric Acid                      2494
Vitamin B12                     953
Uric Acid, Urine                 88
Creatinine ascites               84
Magnesium, Urine                 78
Prolactin                        41
Large Platelets                   1
Name: ITEMID, dtype: int64

In [ ]:
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=(0,15))

KeyboardInterrupt: 

In [ ]:
m_final_lab_med_data

In [10]:
m_labs.to_csv("temp_mimic_{lab_parts}_labs.csv")
m_med1.to_csv("temp_mimic_{lab_parts}_med1.csv")
m_med2.to_csv("temp_mimic_{lab_parts}_med2.csv")

In [11]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=False, lab_parts=(15,30))

Loading med data...
Generate med data....


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:51: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ROW_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")


Generated med data.
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...
Generate lab data from labevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:138: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  labevents = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_LABEVENT_PREPROCESSED))


Generated lab data from labevents.
Generate lab data from chartevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on imp

Generated lab data from chartevents.


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:213: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  merged_chart_lab_events = merged_chart_lab_events.groupby(["HADM_ID", "ADMITTIME", "CHARTTIME", "VALUENUM", "MIMICExtractName"]).nth(0).reset_index()


Loaded Lab data.


In [12]:
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=(15,30))

/Users/pavan/Desktop/TAU/DrugLab/src/modeling/querier.py:119: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  temp = temp.apply(lambda r : self.get_vals(r, t_labs, t_med1, t_med2, before_windows, after_windows), axis=1)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
## Discovery Analysis for the queried medication and lab test pairs in the chosen before and after windows
analyzer = discovery.ClinicalDiscoveryAnalysis(m_final_lab_med_data)
pvals_med_lab = analyzer.analyze(before_windows, after_windows)
sig_med_lab = analyzer.generate_significant(pvals_med_lab.dropna(subset=["TTest Paired"]))

In [ ]:

## Plots
plotter = plots.ClinicalPlotAnalysis(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_corrs_data_df = plotter.plot(m_final_lab_med_data, m_labs, before_windows=before_windows, after_windows=after_windows)